# Lab 3 — clustering

In [66]:
import json
import matplotlib.pyplot as plt
import numpy as np
import pickle

from sklearn.cluster import KMeans

%matplotlib inline
plt.style.use("ggplot")

## 3.11 Clustering Tags

Load the data and prepare the points

In [35]:
with open('data.txt', 'r') as source_file:
    data = json.loads(json.load(source_file)) # Why is this neeeded?

In [68]:
a = {1: 'a', 2:'b'}
[el for el in a]

[1, 2]

In [70]:
tags = [tag for tag in data]
coordinates = np.array([data[tag] for tag in tags])

In [71]:
points

array([[-0.01672448, -0.00824598,  0.03174946,  0.0183414 , -0.00751094],
       [ 0.04401485, -0.04902818,  0.06506781,  0.03154708,  0.0332396 ],
       [ 0.01734856, -0.01683812, -0.0017488 ,  0.04650396,  0.01429893],
       ..., 
       [ 0.01625744, -0.00808403,  0.02321847, -0.012887  ,  0.03012241],
       [ 0.0046428 , -0.00544165,  0.00144497, -0.00364974,  0.00078962],
       [ 0.06868002, -0.04438778,  0.00411947,  0.06091971, -0.0259673 ]])

In [73]:
clus = KMeans(n_clusters=2).fit(points)

array([1, 1, 1, ..., 1, 1, 1], dtype=int32)

In [65]:
tmp

array([[ 0.09881654,  0.07666291,  0.07084682,  0.0364642 ],
       [ 0.10774291,  0.13432533,  0.09306758,  0.08927953],
       [ 0.08864365,  0.08944407,  0.0185628 ,  0.05124579],
       ..., 
       [ 0.06212614,  0.0916594 ,  0.08151897,  0.03706178],
       [ 0.0623801 ,  0.05761464,  0.06398314,  0.01169013],
       [ 0.11150071,  0.10201567,  0.06140767,  0.09665697]])

In [62]:
from bokeh.io import show, output_notebook
from bokeh.plotting import figure, ColumnDataSource
from bokeh.models import HoverTool, ResetTool, PanTool, WheelZoomTool, SaveTool
output_notebook()

# Categorial colors
from bokeh.palettes import Dark2_8
from matplotlib.colors import rgb2hex 

cmap = plt.get_cmap('viridis')
def val2rgb(val):
    return rgb2hex(cmap(val)[:3])

Loading BokehJS ...